A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Load data
df = pd.read_csv('rental_info.csv')

# Calculate the rental length in days
df['rental_length_days'] = (pd.to_datetime(df['return_date']) - pd.to_datetime(df['rental_date'])).dt.days

# Create dummy variables for special features
df["deleted_scenes"] = np.where(df["special_features"].str.contains("Deleted Scenes"), 1, 0)
df["behind_the_scenes"] = np.where(df["special_features"].str.contains("Behind the Scenes"), 1, 0)

cols_to_drop = ["special_features", "rental_length_days", "rental_date", "return_date"]

# Define features (X) and target (y)
X = df.drop(cols_to_drop, axis=1)
y = df["rental_length_days"]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=9)

# Lasso model for feature selection
lasso = Lasso(alpha=1, random_state=9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# Select features with non-zero coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

# Evaluate Linear Regression with Lasso-selected features
ols = LinearRegression()
ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = MSE(y_test, y_test_pred)
print("Linear Regression RMSE: {:.2f}".format(mse_lin_reg_lasso ** 0.5))

# RandomizedSearchCV for Random Forest
param_dist = {
    'n_estimators': np.arange(1, 101, 1),
    'max_depth': np.arange(1, 11, 1)
}
rand_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=9),
    param_distributions=param_dist,
    cv=5,
    random_state=9
)
rand_search.fit(X_lasso_train, y_train)
hyper_params = rand_search.best_params_

# Train Random Forest with best hyperparameters
rf = RandomForestRegressor(
    n_estimators=hyper_params["n_estimators"],
    max_depth=hyper_params["max_depth"],
    random_state=9
)
rf.fit(X_lasso_train, y_train)
rf_pred = rf.predict(X_lasso_test)
mse_rf = MSE(y_test, rf_pred)
print("Random Forest RMSE: {:.2f}".format(mse_rf ** 0.5))

# Compare models and select the best one
if mse_rf < mse_lin_reg_lasso:
    best_model = rf
    best_mse = mse_rf
    print("Best model: Random Forest")
else:
    best_model = ols
    best_mse = mse_lin_reg_lasso
    print("Best model: Linear Regression")

print("Best model RMSE: {:.2f}".format(best_mse ** 0.5))
